## Teak Plantation Volume Estimation with Random Forest and XGBoost

### Import Libraries

In [1]:
import pyodbc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Database Connection and Display the relevant tables in the database

In [2]:
db_path = r"C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb"

conn = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf'DBQ={db_path};'
)

cursor = conn.cursor()

for table in cursor.tables(tableType='TABLE'):
    print(table.table_name)


Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


#### Querying the TreeData table

In [3]:
query = "SELECT * FROM TreeData WHERE AreaType='Teak'"

df = pd.read_sql(query, conn)

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_17596\3977747108.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


#### Data Cleansing and Shaping

In [5]:
df = df.rename(columns={'Tree SPECIES': 'species', 'PLOT': 'plot_id', 'TREE NR': 'tree_id', 'H (m)': 'height_m', 'DBH (cm)': 'dbh_cm'})

In [6]:
df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,plot_id,tree_id,species,Species scientific name,height_m,dbh_cm,Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


In [7]:
df.shape

(354797, 18)

In [8]:
blk_query = r"SELECT * FROM [Block register]"

block_df = pd.read_sql(blk_query, conn)

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_17596\2663950081.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  block_df = pd.read_sql(blk_query, conn)


In [9]:
block_df.head()

,PLANTATION,BLOCK,STATUS,ROTATION,OWNERSHIP,RESERVE,YEAR PLANTED,BLOCK AREA (ha),PRODUCTIVE,UNPRODUCTIVE,TEAK,NATURAL INDIGENOUS,PLANTED INDIGENOUS,GMELINA,OPEN LAND,ROCK,REMARK
0,Akumadan,2A,PLANTED,1,FORM GHANA,ASUBIMA,2008,77.853644,77.853644,0.00,77.853644,NaN,NaN,NaN,NaN,NaN,None
1,Akumadan,2B,PLANTED,1,FORM GHANA,ASUBIMA,2010,4.523337,4.523337,NaN,4.523337,NaN,NaN,NaN,NaN,NaN,None
2,Akumadan,4,PLANTED,1,FORM GHANA,ASUBIMA,2009,42.605816,42.605816,0.00,42.605816,NaN,NaN,NaN,NaN,NaN,None
3,Akumadan,5,PLANTED,1,FORM GHANA,ASUBIMA,2009,63.458447,62.918447,0.54,62.918447,NaN,NaN,NaN,NaN,0.54,None
4,Akumadan,6,PLANTED,1,FORM GHANA,ASUBIMA,2009,78.999204,78.999204,0.00,78.999204,NaN,NaN,NaN,NaN,NaN,None


In [10]:
block_df = block_df[['PLANTATION', 'BLOCK', 'YEAR PLANTED', 'PRODUCTIVE', 'TEAK']]

In [11]:
block_df = block_df.rename(columns={'PLANTATION': 'Plantations', 'BLOCK': 'Block', 'YEAR PLANTED': 'year_planted', 'PRODUCTIVE': 'productive_area', 'TEAK': 'with_teak_ha'})

In [12]:
block_df.head()

,Plantations,Block,year_planted,productive_area,with_teak_ha
0,Akumadan,2A,2008,77.853644,77.853644
1,Akumadan,2B,2010,4.523337,4.523337
2,Akumadan,4,2009,42.605816,42.605816
3,Akumadan,5,2009,62.918447,62.918447
4,Akumadan,6,2009,78.999204,78.999204


In [13]:
plt_blk_relation_query = r"SELECT * FROM [Plot-block relation] WHERE AreaType = 'Teak'"

plt_blk_df = pd.read_sql(plt_blk_relation_query, conn)

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_17596\3021702405.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  plt_blk_df = pd.read_sql(plt_blk_relation_query, conn)


In [14]:
plt_blk_df.head()

,RelationID,Plantations,Plot no,AreaType,Deviating plant year,Block,Start date,End date
0,74,Tain II,1370,Teak,NaN,PF80,2020-05-01,9999-12-31 00:00:00
1,76,Tain II,1372,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
2,77,Tain II,1366,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
3,78,Tain II,1363,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
4,79,Tain II,1374,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00


In [15]:
plt_blk_df = plt_blk_df[['Plantations', 'Plot no', 'AreaType', 'Block']]

In [16]:
plt_blk_df.head()

,Plantations,Plot no,AreaType,Block
0,Tain II,1370,Teak,PF80
1,Tain II,1372,Teak,C17
2,Tain II,1366,Teak,C17
3,Tain II,1363,Teak,C17
4,Tain II,1374,Teak,C17


In [17]:
merged_plt_blk = pd.merge(block_df, plt_blk_df, on=['Plantations', 'Block'])

In [18]:
merged_plt_blk.head()

,Plantations,Block,year_planted,productive_area,with_teak_ha,Plot no,AreaType
0,Akumadan,2A,2008,77.853644,77.853644,667,Teak
1,Akumadan,2A,2008,77.853644,77.853644,666,Teak
2,Akumadan,2A,2008,77.853644,77.853644,4,Teak
3,Akumadan,2A,2008,77.853644,77.853644,5,Teak
4,Akumadan,2A,2008,77.853644,77.853644,8,Teak


In [19]:
merged_plt_blk.shape

(1567, 7)

In [20]:
df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,plot_id,tree_id,species,Species scientific name,height_m,dbh_cm,Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


In [21]:
df = df[['Plantations', 'AreaType', 'Monitoring year', 'plot_id', 'tree_id', 'species', 'height_m', 'dbh_cm', 'REMARKS']]

In [22]:
df.head()

,Plantations,AreaType,Monitoring year,plot_id,tree_id,species,height_m,dbh_cm,REMARKS
0,Tain II,Teak,2018.0,1,71.0,Teak,7.25,12.0,Fire scars
1,Tain II,Teak,2018.0,1,76.0,Teak,8.25,12.0,Fire scars
2,Tain II,Teak,2018.0,1,68.0,Teak,7.00,11.8,Fire scars
3,Tain II,Teak,2018.0,1,53.0,Teak,7.25,11.5,Fire scars
4,Tain II,Teak,2018.0,1,58.0,Teak,7.00,10.5,Fire scars


In [23]:
merged_plt_blk = merged_plt_blk.rename(columns={'Plot no': 'plot_id'})

In [24]:
merged_plt_blk.head()

,Plantations,Block,year_planted,productive_area,with_teak_ha,plot_id,AreaType
0,Akumadan,2A,2008,77.853644,77.853644,667,Teak
1,Akumadan,2A,2008,77.853644,77.853644,666,Teak
2,Akumadan,2A,2008,77.853644,77.853644,4,Teak
3,Akumadan,2A,2008,77.853644,77.853644,5,Teak
4,Akumadan,2A,2008,77.853644,77.853644,8,Teak


In [25]:
df = pd.merge(df, merged_plt_blk, on=['Plantations', 'plot_id', 'AreaType'])

In [26]:
df.head()

,Plantations,AreaType,Monitoring year,plot_id,tree_id,species,height_m,dbh_cm,REMARKS,Block,year_planted,productive_area,with_teak_ha
0,Tain II,Teak,2018.0,1,71.0,Teak,7.25,12.0,Fire scars,B39,2013,61.114539,61.114539
1,Tain II,Teak,2018.0,1,76.0,Teak,8.25,12.0,Fire scars,B39,2013,61.114539,61.114539
2,Tain II,Teak,2018.0,1,68.0,Teak,7.00,11.8,Fire scars,B39,2013,61.114539,61.114539
3,Tain II,Teak,2018.0,1,53.0,Teak,7.25,11.5,Fire scars,B39,2013,61.114539,61.114539
4,Tain II,Teak,2018.0,1,58.0,Teak,7.00,10.5,Fire scars,B39,2013,61.114539,61.114539


In [27]:
df.shape

(344012, 13)

In [28]:
df['age'] = df['Monitoring year'] - df['year_planted']

In [29]:
df.head()

,Plantations,AreaType,Monitoring year,plot_id,tree_id,species,height_m,dbh_cm,REMARKS,Block,year_planted,productive_area,with_teak_ha,age
0,Tain II,Teak,2018.0,1,71.0,Teak,7.25,12.0,Fire scars,B39,2013,61.114539,61.114539,5.0
1,Tain II,Teak,2018.0,1,76.0,Teak,8.25,12.0,Fire scars,B39,2013,61.114539,61.114539,5.0
2,Tain II,Teak,2018.0,1,68.0,Teak,7.00,11.8,Fire scars,B39,2013,61.114539,61.114539,5.0
3,Tain II,Teak,2018.0,1,53.0,Teak,7.25,11.5,Fire scars,B39,2013,61.114539,61.114539,5.0
4,Tain II,Teak,2018.0,1,58.0,Teak,7.00,10.5,Fire scars,B39,2013,61.114539,61.114539,5.0


In [30]:
df['species'].unique()

array(['Teak', 'York', None, 'onyono', 'Mahogany', 'Cassia', 'Albizia',
       'Awiemfosamina', 'Kyenkyen', 'Sofo', 'Onyina', 'Wawa',
       'Unknown indigenous', 'Asanfena', 'Hontan', 'Akuakuo-Ninsuo',
       'Okoro', 'Nyankyerene', 'Watapuo', 'Akonkodie', 'Mango',
       'kwakuanisuo', 'Hyedua', 'Nyamedua', 'kotrenfro', 'Ofram',
       'Konkroma', 'Cedrela', 'Khaya ivorensis', 'Albizia adianthifolia',
       'Kane', 'Wama', 'Wonton', 'Funtumia africana', 'Odum',
       'Trilepisium madagascariense', 'Tetrapleura tetraptera', 'Kusia',
       'Okure', 'Foto', 'Womba', 'Alstonia boonei',
       'Anacardium occidentale', 'Dead', 'Afzelia', 'Otie', 'coconisuol',
       'prekese', 'Kotreamfo', 'Acacia', 'Akye', 'Nofotene', 'Acasia',
       'Kigelia africana', 'Prekese', 'Kookonisuo', 'saiba', 'Pepea',
       '(Cog)Wabire', 'abiziar', 'Pampam', 'Sese', 'Sybie', 'Cola',
       'Palm tree', 'Saiba', 'Indigenous', 'indigenous'], dtype=object)

In [ ]:
teak = 'Teak'

for i in df['species']:
    if i != teak:
        df = df[~((df['species'] == i))]

In [ ]:
df['species'].unique()

In [ ]:
df.shape

In [ ]:
df['Plantations'].unique()

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=['height_m', 'dbh_cm'])

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df['REMARKS'].unique()

In [ ]:
unwanted_list = ['Beetle infestation', 'Top broken', 'Coppice', 'Broken top', 'Forked', 'Dead tree', 'Forked at 30 cm', 'Fork at 2m', 'Nearly dead',
                  'Wildling', 'Outside planting line - Wildling', 'marked', 'Dead tree - mared', 'dead tree, but resprouting', 'Sapling - Wildling', 
                 'dead tree, but coppicing', 'Old teak stand', 'Dying', 'ceber', 'Main tree dead', 'Top dead', 'Forked at DBH, taken above', 'Top dead; monitoring day unknown',
                 'Diseased', 'Teak dieback', 'Top dead - Wildling', 'Old teak - Coppice', 'Old teak - Teak dieback', 'Old teak', 'Burnt - Wildling', 'coppice; monitoring day unknown',
                 'diseased - Diameter taken with diameter tape', 'diseased - Diameter taken with diameter tape', 'Diameter taken with diameter tape - Wildling', 'Tape - Wildling',
                 'Slightly dead. Diameter taken with diameter tape - marked', 'Half dead. Diameter taken with diameter tape; monitoring day unknown'
                ]

for i in unwanted_list:
    df = df[~((df['REMARKS'] == i))]

In [ ]:
df['REMARKS'].unique()

In [ ]:
not_needed_list = ['dead', 'Coppice, Marked', 'forked', 'Marked', 'dead,marked', 'Abnormal tree,correct measurements', 'coppice, marked', 'coppice', 'wildling',
                  'wildling, marked', 'marked,half dead,top broken', 'DEAD', ' dead', 'Half Dead']

for i in not_needed_list:
    df = df[~((df['REMARKS'] == i))]

In [ ]:
df.shape

## Compute Tree Volumes

In [ ]:
df['basal_area'] = (np.pi / 40000) * (df['dbh_cm']**2)

In [ ]:
df.head()

In [ ]:
form_factor = 0.45 # replace with the actual form factor
df['volume'] = df['basal_area'] * df['height_m'] * form_factor

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
thin_hist_query = r"SELECT * FROM [Thinning history] WHERE AreaType = 'Teak'"

thin_hist_df = pd.read_sql(thin_hist_query, conn)

In [ ]:
thin_hist_df.head()

In [ ]:
thin_hist_df = thin_hist_df[['Plantation', 'AreaType', 'Block', 'Thinning number']]

In [ ]:
thin_hist_df = thin_hist_df.rename(columns={'Plantation': 'Plantations'})

In [ ]:
thin_hist_df.head()

In [ ]:
df = pd.merge(df, thin_hist_df, on=['Plantations', 'AreaType', 'Block'])

In [ ]:
df.head()

#### Visualized Data

In [ ]:
import seaborn as sns

sns.pairplot(df, vars=['dbh_cm', 'height_m', 'age', 'volume'])
plt.show()

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Thinning number'], drop_first=True)

In [ ]:
df_encoded.head()

In [ ]:
from sklearn.model_selection import train_test_split

features = ['dbh_cm', 'height_m', 'age', 'Thinning number_T2', 'Thinning number_T3', 'Thinning number_T4']

X = df_encoded[features]
y = df_encoded['volume']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

### Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

### XGBoost Model

In [ ]:
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=100, random_state=42)
model_xgb.fit(X_train, y_train)

### Models Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

for name, model in [('Linear', model_lr), ('RandomForest', model_rf), ('XGBoost', model_xgb)]:
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name}: MSE={mse:.2f}, R2={r2:.2f}")

### Visualize Predictions Vs. Actual

In [ ]:
plt.scatter(y_test, model_rf.predict(X_test))
plt.xlabel('Actual Volume')
plt.ylabel('Predicted Volume')
plt.title('Random Forest: Actual vs Predicted')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

In [ ]:
plt.scatter(y_test, model_xgb.predict(X_test))
plt.xlabel('Actual Volume')
plt.ylabel('Predicted Volume')
plt.title('XGBoost: Actual vs Predicted')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')

In [ ]:
plt.scatter(y_test, model_lr.predict(X_test))
plt.xlabel('Actual Volume')
plt.ylabel('Predicted Volume')
plt.title('Linear Regression: Actual Vs Predicted')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.show()

### Feature Importance

In [ ]:
importances = model_rf.feature_importances_

for feat, imp in zip(features, importances):
    print(f"{feat}: {imp:.2f}")

In [ ]:
sns.barplot(x=importances, y=features)
plt.title('Feature Importance')
plt.show()

### Predict New Data

In [ ]:
new_tree = pd.DataFrame({'dbh_cm': [30], 'height_m': [25], 'age': [15], 'Thinning number_T2': [True], 'Thinning number_T3': [True], 'Thinning number_T4': [False]})
predicted_volume = model_rf.predict(new_tree)
print(f"Predicted volume: {predicted_volume[0]:.2f} m3")